# Set-up

In [ ]:
# Define the base input directory
base_data_path <- '/users/genomics/xoel/Review_CD/ewce_linnarson.v7/data/'
in.dir <- base_data_path

In [2]:
library(glue)
library(Matrix)
library(spam)
library(spam64)
library(dplyr)

Spam version 2.10-0 (2023-10-23) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


Attaching package: ‘spam’


The following object is masked from ‘package:Matrix’:

    det


The following objects are masked from ‘package:base’:

    backsolve, forwardsolve



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# **Part 1: List the folders to process**

In [3]:
group_vars = c('Subregion', 'Age')

folders <- list()

region_dirs <- list.dirs(glue('{in.dir}/by_{group_vars[1]}/'), full.names = T, recursive = F)

for (region_dir in region_dirs){
    
    region <- basename(region_dir)
    
    age_dirs <- paste0(region_dir, '/by_', group_vars[2], '/') %>%
    list.dirs(full.names = T, recursive = F)
    
    for (age_dir in age_dirs){
        age <- basename(age_dir)

        d <- data.frame(list(region, age, normalizePath(age_dir)))
        colnames(d) <- c(group_vars, 'path')
        folders <- c(folders, list(d))
    }
}

folders <- do.call(folders, what = 'rbind')
folders$size <- paste('cd', folders$path, '; du -s ./') %>% sapply(system, intern = T) %>% gsub(pattern = '\t./', replacement = '', fixed=T) %>% as.numeric
folders <- folders %>% arrange(size)

# **Part 2: Load gene metadata**

In [ ]:
gene.file <- glue('{in.dir}GeneInfo.csv')
Ginfo <- data.table::fread(gene.file, data.table=F)[,-1]
sum(duplicated(Ginfo$Gene))
Ginfo$Name <- make.unique(Ginfo$Gene)
sum(duplicated(Ginfo$Name))

[1] 1059

[1] 0

In [5]:
dim(Ginfo)

[1] 59480    11

# **Part 3: Helper functions**

In [6]:
options(future.globals.maxSize = 500*(1024**3))

In [7]:
# A function to normalize a expression matrix following EWCE functions
my.sct_normalize <- function (exp, verbose = TRUE) {
    
    requireNamespace("sctransform")
    
    if (verbose){print(dim(exp))}    
    gc()
    exp <- sctransform::vst(umi = exp, 
                            return_cell_attr = F, 
                            return_gene_attr = F, 
                            return_corrected_umi = T, 
                            verbosity = if (verbose) {2} else {0})$umi_corrected
    
    
    if (verbose){message(glue('[{Sys.time()}] MaxScaling...'))}
    exp <- Matrix::t(Matrix::t(exp) * (1/Matrix::colSums(exp)))
    if (verbose){print(dim(exp))}
    return(exp)
}

my_generate_ctd <- function(sct.exp, 
                            level1class, level2class = NULL,
                            as_sparse=F, 
                            as_DelayedArray = F,
                            input_species = 'hs', 
                            savePath = './',
                            groupName = 'CTD_expression',
                            no_cores = parallel::detectCores(),
                            return_ctd = F,
                            verbose = T){
    
    
    annotLevels <- list(level1class=level1class)
    if (!is.null(level2class)){
        annotLevels$level2class <- level2class
    }

    annotLevels <- lapply(names(annotLevels), function(n){
        cs <- annotLevels[[n]]
        print(length(cs))
        if (length(cs) != dim(sct.exp)[2]){
            stop(glue('Length of {n} {length(cs)} is not the same as expression matrix {dim(sct.exp)[2]}'))
        }
        if (!is.factor(cs)){
            cs <- factor(as.character(cs))
        }
        if (length(unique(cs)) == 1){
            print(glue('Number of unique groups in {n} is 1. It will not be used.'))
            return(NULL)
        }
        return(cs)
    }) %>% setNames(nm = names(annotLevels))
    annotLevels <- annotLevels[!sapply(annotLevels,is.null)]

    
    print(lapply(annotLevels, head))
    dir.create(savePath, showWarnings = F)
    
    # Generate celltype data
    CTD <- EWCE::generate_celltype_data(
        exp = sct.exp,
        annotLevels = annotLevels,
        as_sparse=as_sparse,
        as_DelayedArray=as_DelayedArray,
        input_species=input_species,
        savePath=savePath,
        groupName=groupName,
        no_cores=no_cores,
        return_ctd=return_ctd,
        verbose=verbose
    )
    return(CTD)
}


# A function to load a expression matrix, subset it, normalize it and save it
ctd_from_exp_path <- function(exp_path, cellmeta_path, ctd_folder, 
                              overwrite = F,
                              level1_var, level2_var = NULL, 
                              cellname_column = 'CellID', 
                              row.names = NULL, return_ctd = F,
                              verbose = T, ...){
    

    requireNamespace("Matrix")

    
    ctd_name <- c(level1_var, level2_var)
    ctd_name <- paste0('_', paste0(ctd_name[!is.null(ctd_name)], collapse = '.'))
    if (verbose){message(glue('[{Sys.time()}] Using name {ctd_name}'))}


    if (verbose){message(glue('[{Sys.time()}] Loading cell metadata...'))}
    cellmeta <- data.table::fread(cellmeta_path, nThread = parallel::detectCores(), data.table = F)

    if (verbose){message(glue('[{Sys.time()}] Retrieving classes...'))}
    level1class <- cellmeta[,level1_var, drop=T]
    level2class <- if(is.null(level2_var)){NULL}else{cellmeta[,level2_var, drop=T]}
    
    if (verbose){message(glue('[{Sys.time()}] Loading file...'))}
    mtx <- as(t(Matrix::readMM(exp_path)), 'CsparseMatrix')
    
    if (verbose){message(glue('[{Sys.time()}] Setting dimnames...'))}
    colnames(mtx) <- if(is.null(cellname_column)){as.character(1:nrow(mtx))}else{cellmeta[,cellname_column]}
    rownames(mtx) <- if(is.null(row.names)){as.character(1:nrow(mtx))}else{row.names}
    
    if (verbose){message(glue('[{Sys.time()}] Normalizing...'))}
    gc()
    mtx <- my.sct_normalize(mtx, verbose=verbose)

    gc()
    return(
        my_generate_ctd(
            sct.exp = mtx, 
            level1class=level1class, 
            level2class=level2class,
            savePath = ctd_folder, groupName = ctd_name,
            return_ctd = return_ctd, verbose = verbose)
    )
}

# **Part 4: Process folders**

In [8]:
# names of rows (Genes)
row.names <- Ginfo$Name

level1var <- 'ShortName'
level2var <- NULL


In [9]:
for (row in 1:nrow(folders)){
    path <- folders[row, 'path']
    print(glue('{row}/{nrow(folders)}'))
    print(folders[row,])
    
    cellmeta_path <- paste0(path, '/cell_meta.csv')
    expression_path <- paste0(path, '/Expression.sparseMatrix.mtx')

    ctd_from_exp_path(exp_path=expression_path, 
                      cellmeta_path=cellmeta_path,
                      ctd_folder=path, 
                      overwrite = F,
                      level1_var=level1var, level2_var=level2var, 
                      cellname_column = 'CellID', 
                      row.names = row.names, return_ctd = F,
                      verbose = T)
}

1/29
      Subregion Age
1 Telencephalon 6.7
                                                              path   size
1 /scratch/xoel/ewce_data.v7/by_Subregion/Telencephalon/by_Age/6.7 231588


[2024-11-28 14:58:47.123345] Using name _ShortName

[2024-11-28 14:58:47.127428] Loading cell metadata...

[2024-11-28 14:58:47.215291] Retrieving classes...

[2024-11-28 14:58:47.217697] Loading file...

[2024-11-28 14:58:54.294888] Setting dimnames...

[2024-11-28 14:58:54.297467] Normalizing...

Loading required namespace: sctransform



[1] 59480  6903


Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 24419 by 6903

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 6903 cells



  |======================================================================| 100%


...

Computing corrected count matrix for 37851 genes



  |======================================================================| 100%


Wall clock passed: Time difference of 13.96203 mins

[2024-11-28 17:29:50.57417] MaxScaling...



[1] 37851 85160
[1] 85160
$level1class
[1] Imm Imm Imm Imm Imm Imm
Levels: ExNeu ExNeuBl ExNeuIPC Imm InNeu InNeuBl InNeuIPC Pery RGC



96 core(s) assigned as workers (0 reserved).

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 24.0 GiB”
+ Calculating normalized mean expression.

Converting to sparse matrix.

+ Calculating normalized specificity.

Converting to sparse matrix.

Converting to sparse matrix.

+ Saving results ==>  /scratch/xoel/ewce_data.v7/by_Subregion/Brain/by_Age/6.0/ctd__ShortName.rda

